In [7]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from glob import glob
from PIL import Image
from sklearn.model_selection import train_test_split






In [8]:

random.seed(42)
np.random.seed(42)
torch.manual_seed(42)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [9]:
class OmniglotDataset(Dataset):
    def __init__(self, data_paths, targets):
        self.data = data_paths
        self.targets = targets

    def __getitem__(self, index):
        img_path = self.data[index]
        target = self.targets[index]

        # Loading image
        img = Image.open(img_path).convert("L")  # Converting to grayscale
        img = np.array(img, dtype=np.float32) / 255.0  # Normalizing
        img = torch.tensor(img).unsqueeze(0).float()
        return img, target

    def __len__(self):
        return len(self.data)

# 2. Function to load and split the dataset
def prepare_omniglot_dataset(root_dir, test_size=0.2):
    """
    Load Omniglot dataset and split it into train and test sets

    Args:
        root_dir: Path to the Omniglot dataset (should contain 'images_background' or similar)
        test_size: Proportion of data to use for test set

    Returns:
        train_dataset, test_dataset
    """
    print(f"Loading Omniglot dataset from: {root_dir}")

    # Finding all image files and their paths
    all_images = []


    if os.path.exists(os.path.join(root_dir, 'images_background')):

        background_images = glob(os.path.join(root_dir, 'images_background', '*', '*', '*.png'))
        all_images.extend(background_images)


        if os.path.exists(os.path.join(root_dir, 'images_evaluation')):
            evaluation_images = glob(os.path.join(root_dir, 'images_evaluation', '*', '*', '*.png'))
            all_images.extend(evaluation_images)
    else:

        all_images = glob(os.path.join(root_dir, '*', '*', '*.png'))

    if not all_images:
        raise ValueError(f"No images found in {root_dir}. Please check the path.")

    print(f"Found {len(all_images)} images total")


    label_map = {}
    targets = []

    for path in all_images:
        character_dir = os.path.dirname(path)
        if character_dir not in label_map:
            label_map[character_dir] = len(label_map)
        targets.append(label_map[character_dir])

    # Split into train and test sets
    train_paths, test_paths, train_targets, test_targets = train_test_split(
        all_images, targets, test_size=test_size, stratify=targets, random_state=42
    )

    print(f"Split dataset into {len(train_paths)} training images and {len(test_paths)} test images")
    print(f"Number of unique classes: {len(label_map)}")

    # Create datasets
    train_dataset = OmniglotDataset(train_paths, train_targets)
    test_dataset = OmniglotDataset(test_paths, test_targets)

    return train_dataset, test_dataset


In [10]:
class SiameseOmniglotDataset(torch.utils.data.Dataset):
    def __init__(self, dataset):
        self.dataset = dataset
        self.labels_to_indices = self._group_by_label()  # Group indices by label

    def _group_by_label(self):
        """ Groups dataset indices by their labels. """
        labels_to_indices = {}
        for i in range(len(self.dataset)):
            _, label = self.dataset[i]
            if label not in labels_to_indices:
                labels_to_indices[label] = []
            labels_to_indices[label].append(i)
        return labels_to_indices

    def __getitem__(self, index):
        """ Returns a pair of images (same or different class) and a label. """
        img1, label1 = self.dataset[index]

        if random.random() < 0.5:

            img2 = self.dataset[random.choice(self.labels_to_indices[label1])][0]
            label = 1
        else:

            label2 = random.choice(list(self.labels_to_indices.keys()))
            while label2 == label1:
                label2 = random.choice(list(self.labels_to_indices.keys()))
            img2 = self.dataset[random.choice(self.labels_to_indices[label2])][0]
            label = 0

        return img1, img2, torch.tensor(label, dtype=torch.float32)

    def __len__(self):
        return len(self.dataset)


In [11]:
class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d((6, 6))
        )
        self.fc = nn.Sequential(
            nn.Linear(256 * 6 * 6, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 128)
        )

    def forward_once(self, x):
        x = self.cnn(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

    def forward(self, x1, x2):
        out1 = self.forward_once(x1)
        out2 = self.forward_once(x2)
        return out1, out2


In [12]:
class ContrastiveLoss(nn.Module):
    def __init__(self, margin=2.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, out1, out2, label):
        distance = F.pairwise_distance(out1, out2, keepdim=True)
        loss = (1 - label) * distance.pow(2) + label * F.relu(self.margin - distance).pow(2)
        return loss.mean()



In [13]:
def train(model, train_loader, criterion, optimizer, epochs=5):
    model.train()
    for epoch in range(epochs):
        epoch_loss = 0
        for batch_idx, (img1, img2, labels) in enumerate(train_loader):
            img1, img2, labels = img1.to(device), img2.to(device), labels.to(device)

            optimizer.zero_grad()
            out1, out2 = model(img1, img2)
            loss = criterion(out1, out2, labels)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()


            if batch_idx % 50 == 0:
                print(f"Epoch {epoch+1}/{epochs} | Batch {batch_idx}/{len(train_loader)} | Loss: {loss.item():.4f}")

        print(f"Epoch [{epoch+1}/{epochs}], Average Loss: {epoch_loss/len(train_loader):.4f}")


def evaluate(model, test_loader, criterion):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for img1, img2, labels in test_loader:
            img1, img2, labels = img1.to(device), img2.to(device), labels.to(device)


            out1, out2 = model(img1, img2)


            distance = F.pairwise_distance(out1, out2)


            loss = criterion(out1, out2, labels)
            total_loss += loss.item()


            predictions = (distance < 1.0).float()

            correct += (predictions == labels).sum().item()
            total += labels.size(0)

    accuracy = correct / total
    avg_loss = total_loss / len(test_loader)
    print(f"Test Loss: {avg_loss:.4f}, Test Accuracy: {accuracy * 100:.2f}%")
    return accuracy

In [14]:
def run_siamese_network(dataset_path, batch_size=32, epochs=10, test_size=0.2):
    """
    Main function to prepare dataset, train and evaluate the Siamese Network

    Args:
        dataset_path: Path to Omniglot dataset
        batch_size: Batch size for training
        epochs: Number of training epochs
        test_size: Proportion of data to use for testing
    """
    # Preparing datasets
    train_dataset, test_dataset = prepare_omniglot_dataset(dataset_path, test_size)

    # Creating Siamese datasets
    train_siamese_dataset = SiameseOmniglotDataset(train_dataset)
    test_siamese_dataset = SiameseOmniglotDataset(test_dataset)

    # Creating data loaders
    train_loader = DataLoader(train_siamese_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_siamese_dataset, batch_size=batch_size, shuffle=False)

    print(f"Prepared dataloaders with batch size {batch_size}")

    # Initializing model, loss and optimizer
    model = SiameseNetwork().to(device)
    criterion = ContrastiveLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Training the model
    print("Starting training...")
    train(model, train_loader, criterion, optimizer, epochs=epochs)

    # Evaluating the model
    print("Evaluating model...")
    accuracy = evaluate(model, test_loader, criterion)

    # Saving the model
    save_path = "/content/siamese_omniglot_model.pth"
    torch.save(model.state_dict(), save_path)
    print(f"Model saved to {save_path}")

    return model, accuracy


dataset_path = "/content/drive/MyDrive/Few-Shot Learning for Handwritten Character Recognition/images_background"

# Running the Siamese Network
model, accuracy = run_siamese_network(
    dataset_path=dataset_path,
    batch_size=32,
    epochs=15,
    test_size=0.2
)

Loading Omniglot dataset from: /content/drive/MyDrive/Few-Shot Learning for Handwritten Character Recognition/images_background
Found 385 images total
Split dataset into 308 training images and 77 test images
Number of unique classes: 20
Prepared dataloaders with batch size 32
Starting training...
Epoch 1/15 | Batch 0/10 | Loss: 1.6157
Epoch [1/15], Average Loss: 1.4956
Epoch 2/15 | Batch 0/10 | Loss: 1.0204
Epoch [2/15], Average Loss: 1.3424
Epoch 3/15 | Batch 0/10 | Loss: 1.6032
Epoch [3/15], Average Loss: 1.2991
Epoch 4/15 | Batch 0/10 | Loss: 1.1731
Epoch [4/15], Average Loss: 1.2412
Epoch 5/15 | Batch 0/10 | Loss: 1.2108
Epoch [5/15], Average Loss: 1.1567
Epoch 6/15 | Batch 0/10 | Loss: 1.1235
Epoch [6/15], Average Loss: 1.2103
Epoch 7/15 | Batch 0/10 | Loss: 1.2656
Epoch [7/15], Average Loss: 1.1571
Epoch 8/15 | Batch 0/10 | Loss: 1.1310
Epoch [8/15], Average Loss: 1.1543
Epoch 9/15 | Batch 0/10 | Loss: 1.1458
Epoch [9/15], Average Loss: 1.1215
Epoch 10/15 | Batch 0/10 | Loss: 1.